# Read the data

In [1]:
import os
import pandas as pd

from fastapi import FastAPI
from typing import List
from models import CountryData, Pillar, Area, Indicator, Metric
import re

/Users/jessiefung/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
filepath = "../data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx" # Specify the correct path to the file
df_assessments = pd.read_excel(filepath)

# Convert the date columns to datetime type so we can filter by year later
df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])

/var/folders/kv/l9vcg1gs7xb1tfhh37wfc4_w0000gn/T/ipykernel_63610/3951065428.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])


# Define the function

In [3]:
country = 'Barbados'
assessment_year = 2024

data = df_assessments[(df_assessments["Country"] == country) & (df_assessments["Assessment date"].dt.year == assessment_year)]

#remember which columns are area, indicator, metric
area_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("area")]
indicator_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("indicator")]
metric_cols =  [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("metric")] 

#remove unecessary columns
data = data[[col for col in data.columns if col.startswith(("area", "indicator", "metric"))]]

#rename columns so they align with output
remap_column_names = {col: re.sub(".*?\s", "", col) for col in data.columns}
data = data.rename(columns=remap_column_names)


#get flat Pandas series of country data
data = data.iloc[0]
data = data.fillna("")

data

EP.1                             Yes
EP.1.a                           Yes
EP.1.b                       No Data
EP.1.c                           Yes
EP.2                         Partial
                      ...           
CF.4                                
CF.4.i      10.95 MW/US$ billion GDP
CF.4.ii      4.69 MW/US$ billion GDP
CF.4.iii                     No data
CF.4.iv                      No data
Name: 32, Length: 74, dtype: object

In [6]:
#get metric
metrics = [{'name': metric, 'value': data[f'{metric}']} for metric in metric_cols]

#get indicator
indicators = [{'name': indicator, 'assessment': data[f"{indicator}"],
                   'metrics': next((met for met in metrics if met["name"].startswith(indicator)), "")} for indicator in indicator_cols]   

    #get area
areas = [{'name': area, 'assessment': data[f"{area}"],
              'indicators': [ind for ind in indicators if ind["name"].startswith(area)]} for area in area_cols]


#get pillar
pillars = [{'name': pillar, 'areas': areas} for pillar in ["EP","CP","CF"]]

output_dict = {'metadata': {'country': country, 'assessment_year': assessment_year},
                'pillars': [pillar for pillar in pillars]}

output_dict

{'metadata': {'country': 'Barbados', 'assessment_year': 2024},
 'pillars': [{'name': 'EP',
   'areas': [{'name': 'EP.1',
     'assessment': 'Yes',
     'indicators': [{'name': 'EP.1.a', 'assessment': 'Yes', 'metrics': ''},
      {'name': 'EP.1.b', 'assessment': 'No Data', 'metrics': ''},
      {'name': 'EP.1.c', 'assessment': 'Yes', 'metrics': ''}]},
    {'name': 'EP.2',
     'assessment': 'Partial',
     'indicators': [{'name': 'EP.2.a',
       'assessment': 'Yes',
       'metrics': {'name': 'EP.2.a.i', 'value': '-73%'}},
      {'name': 'EP.2.b',
       'assessment': 'No',
       'metrics': {'name': 'EP.2.b.i',
        'value': 'No or unsuitable disclosure'}},
      {'name': 'EP.2.c',
       'assessment': 'No Data',
       'metrics': {'name': 'EP.2.c.i', 'value': 'No Data'}},
      {'name': 'EP.2.d',
       'assessment': 'Yes',
       'metrics': {'name': 'EP.2.d.i', 'value': '-94%'}}]},
    {'name': 'EP.3',
     'assessment': 'Yes',
     'indicators': [{'name': 'EP.3.a',
       'asses

In [7]:
CountryData(**output_dict)

CountryData(metadata=Metadata(country='Barbados', assessment_year=2024), pillars=[Pillar(name='EP', areas=[Area(name='EP.1', assessment='Yes', indicators=[Indicator(name='EP.1.a', assessment='Yes', metrics=''), Indicator(name='EP.1.b', assessment='No Data', metrics=''), Indicator(name='EP.1.c', assessment='Yes', metrics='')]), Area(name='EP.2', assessment='Partial', indicators=[Indicator(name='EP.2.a', assessment='Yes', metrics=Metric(name='EP.2.a.i', value='-73%')), Indicator(name='EP.2.b', assessment='No', metrics=Metric(name='EP.2.b.i', value='No or unsuitable disclosure')), Indicator(name='EP.2.c', assessment='No Data', metrics=Metric(name='EP.2.c.i', value='No Data')), Indicator(name='EP.2.d', assessment='Yes', metrics=Metric(name='EP.2.d.i', value='-94%'))]), Area(name='EP.3', assessment='Yes', indicators=[Indicator(name='EP.3.a', assessment='Yes', metrics=Metric(name='EP.3.a.i', value='2030')), Indicator(name='EP.3.b', assessment='Yes', metrics=''), Indicator(name='EP.3.c', asse

In [8]:
def get_country_data(country: str, assessment_year: int):
    data = df_assessments[(df_assessments["Country"] == country) & (df_assessments["Assessment date"].dt.year == assessment_year)]

    #remember which columns are area, indicator, metric
    area_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("area")]
    indicator_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("indicator")]
    metric_cols =  [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("metric")] 

    #remove unecessary columns
    data = data[[col for col in data.columns if col.startswith(("area", "indicator", "metric"))]]

    #rename columns so they align with output
    remap_column_names = {col: re.sub(".*?\s", "", col) for col in data.columns}
    data = data.rename(columns=remap_column_names)

    #get flat Pandas series of country data
    data = data.iloc[0]
    data = data.fillna("")

    #get metric
    metrics = [{'name': metric, 'value': data[f'{metric}']} for metric in metric_cols]

    #get indicator
    indicators = [{'name': indicator, 'assessment': data[f"{indicator}"],
                   'metrics': next((met for met in metrics if met["name"].startswith(indicator)), "")} for indicator in indicator_cols]   

    #get area
    areas = [{'name': area, 'assessment': data[f"{area}"],
              'indicators': [ind for ind in indicators if ind["name"].startswith(area)]} or "" for area in area_cols]

    #get pillar
    pillars = [{'name': pillar, 'areas': areas} for pillar in ["EP","CP","CF"]]

    output_dict = {'metadata': {'country': country, 'assessment_year': assessment_year},
                   'pillars': [pillar for pillar in pillars]}

    output = CountryData(**output_dict)

    return output

In [9]:
unique_values = [df_assessments[col].unique().tolist() for col in df_assessments.columns if col.startswith(("metric"))]

In [10]:
list((set(sum(unique_values, []))))

['0.96%',
 '205.87 MW/US$ billion GDP',
 '-32%',
 '681%',
 '140%',
 '-41%',
 '2040',
 '5.81 MJ/PPP-adjusted GDP',
 '83%',
 '-23%',
 '32.93 US$/tCO2e',
 '-44%',
 '220%',
 '13.63 MW/US$ billion GDP',
 '17%',
 '0.21%',
 '19%',
 '563%',
 '2.24 MW/US$ billion GDP',
 '931%',
 '1.88%',
 '57%',
 '31.20 MW/US$ billion GDP',
 '0.92',
 '1.85 MW/US$ billion GDP',
 '807%',
 '2060',
 '30%',
 '6.22%',
 '609%',
 '0.06%',
 '1.14 MW/US$ billion GDP',
 '55%',
 '7.93 US$/tCO2e',
 '429.46 MW/US$ billion GDP',
 '2.74 MJ/US$ PPP-adjusted GDP',
 '1.02 MW/US$ billion GDP',
 '2.48 MJ/US$ PPP-adjusted GDP',
 '22.10 MW/US$ billion GDP',
 '-30%',
 '7.97 MW/US$ billion GDP',
 'US$ 48.50/tCO₂e',
 '5.02 MJ/PPP-adjusted GDP',
 '0.25 MW/US$ billion GDP',
 '0.75',
 '2.31 MJ/PPP-adjusted GDP',
 '865%',
 '0.03%',
 '0.85',
 '1.01%',
 '1.05 MW/US$ billion GDP',
 '16%',
 '3.72 MW/US$ billion GDP',
 '3.18 MJ/US$ PPP-adjusted GDP',
 '15.15 MW/US$ billion GDP',
 '75%',
 '2.60%',
 '13.24 MW/US$ billion GDP',
 '0.22%',
 '1188%',


# Test how the function behaves

In [11]:
get_country_data("Barbados", 2024)

CountryData(metadata=Metadata(country='Barbados', assessment_year=2024), pillars=[Pillar(name='EP', areas=[Area(name='EP.1', assessment='Yes', indicators=[Indicator(name='EP.1.a', assessment='Yes', metrics=''), Indicator(name='EP.1.b', assessment='No Data', metrics=''), Indicator(name='EP.1.c', assessment='Yes', metrics='')]), Area(name='EP.2', assessment='Partial', indicators=[Indicator(name='EP.2.a', assessment='Yes', metrics=Metric(name='EP.2.a.i', value='-73%')), Indicator(name='EP.2.b', assessment='No', metrics=Metric(name='EP.2.b.i', value='No or unsuitable disclosure')), Indicator(name='EP.2.c', assessment='No Data', metrics=Metric(name='EP.2.c.i', value='No Data')), Indicator(name='EP.2.d', assessment='Yes', metrics=Metric(name='EP.2.d.i', value='-94%'))]), Area(name='EP.3', assessment='Yes', indicators=[Indicator(name='EP.3.a', assessment='Yes', metrics=Metric(name='EP.3.a.i', value='2030')), Indicator(name='EP.3.b', assessment='Yes', metrics=''), Indicator(name='EP.3.c', asse

# Testing out Pydantic Models

In [9]:
from pydantic import BaseModel, Field
from typing import Literal